In [55]:
import pandas as pd
import os
import numpy as np


Simple Moving Average (SMA): The average of a stock's closing prices over a specified period. It helps identify overall trends and potential support and resistance levels.

Exponential Moving Average (EMA): Similar to SMA but with more weight given to recent prices. It reacts more quickly to price changes and provides a smoother curve.

Relative Strength Index (RSI): Measures the speed and change of price movements to determine overbought or oversold conditions. Ranges between 0 and 100, with values above 70 indicating overbought and below 30 indicating oversold.

Moving Average Convergence Divergence (MACD): Compares two moving averages (usually 12-day and 26-day EMAs) to identify potential trend reversals and momentum shifts.

Bollinger Bands: A volatility indicator that consists of a middle band (SMA) and two outer bands placed at a specified standard deviation. Widening bands suggest increased volatility, while narrowing bands indicate decreased volatility.

Average True Range (ATR): Measures market volatility by calculating the average of true price ranges over a specified period. A higher ATR indicates higher volatility.

Stochastic Oscillator: Compares a stock's closing price to its price range over a specified period. It helps identify potential trend reversals and overbought/oversold conditions.

On-Balance Volume (OBV): A cumulative indicator that adds volume on up days and subtracts volume on down days. It helps determine the strength of a price trend by analyzing changes in trading volume.

Chaikin Money Flow (CMF): Measures the amount of money flowing in and out of a stock over a specified period. A positive CMF indicates buying pressure, while a negative CMF suggests selling pressure.

Parabolic SAR: A trend-following indicator that plots a series of dots on a chart to identify potential trend reversals and stop-loss levels.

Ichimoku Cloud: A comprehensive indicator that combines multiple moving averages, support and resistance levels, and trend lines to provide a holistic view of the market.

Fibonacci Retracements: A tool based on the Fibonacci sequence that identifies potential support and resistance levels by measuring the percentage retracements of a previous price movement.

Pivot Points: A technical analysis tool that calculates potential support and resistance levels using the high, low, and closing prices of the previous trading period.

Rate of Change (ROC): Measures the percentage change in price over a specified period. It helps identify momentum and potential trend reversals.

Commodity Channel Index (CCI): Compares the current price to its average price over a specified period. Used to identify overbought and oversold conditions.

Volume Weighted Average Price (VWAP): Calculates the average price of a stock based on both price and trading volume. Often used by institutional traders to gauge market liquidity and execution quality.

Accumulation/Distribution Line (ADL): Analyzes the relationship between price and volume to determine if a stock is being accumulated or distributed. A rising ADL indicates accumulation, while a falling ADL suggests distribution.

Donchian Channels: A trend-following indicator that plots the highest high and lowest low over a specified period. Used to identify potential breakouts and trend reversals.

Keltner Channels: Similar



In [56]:
def RSI(df, n=14):
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(n).mean()
    avg_loss = loss.rolling(n).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return delta, avg_gain, avg_loss , rsi
def SMA(df, column, period):
    return df[column].rolling(period).mean()
def EMA(df, column, period):
    try:
        return df[column].ewm(span=period, adjust=False).mean()
    except KeyError:
        return df.ewm(span=period, adjust=False).mean()
def WMA(df, column, period):
    weights = np.arange(1, period+1)
    wma = df[column].rolling(period).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
    return wma
def Bollinger_Bands(df, column, period, std):
    sma = SMA(df, column, period)
    std_dev = df[column].rolling(period).std()
    upper_band = sma + (std * std_dev)
    lower_band = sma - (std * std_dev)
    return sma, upper_band, lower_band
def MACD(df, column, fast_period, slow_period, signal_period):
    ema_fast = EMA(df, column, fast_period)
    ema_slow = EMA(df, column, slow_period)
    macd_line = ema_fast - ema_slow
    signal_line = EMA(macd_line, column, signal_period)
    return macd_line, signal_line

def Stochastic_Oscillator(df, high_col, low_col, close_col, n=14):
    highest_high = df[high_col].rolling(n).max()
    lowest_low = df[low_col].rolling(n).min()
    k = ((df[close_col] - lowest_low) / (highest_high - lowest_low)) * 100
    return k

def ATR(df, high_col, low_col, close_col, n=14):
    tr1 = abs(df[high_col] - df[low_col])
    tr2 = abs(df[high_col] - df[close_col].shift())
    tr3 = abs(df[low_col] - df[close_col].shift())
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = true_range.rolling(n).mean()
    return atr

def ROC(df, column, n=12):
    roc = 100 * (df[column] / df[column].shift(n) - 1)
    return roc


def OBV(df, close_col, volume_col):
    change_in_volume = np.where(df[close_col] > df[close_col].shift(1), df[volume_col], np.where(df[close_col] < df[close_col].shift(1), -df[volume_col], 0))
    obv = change_in_volume.cumsum()
    return obv

def RVI(df, close_col, high_col, low_col, n=14):
    diff = df[close_col].diff(1)
    up_chg = 0 * diff
    down_chg = 0 * diff
    up_chg[diff > 0] = diff[diff > 0]
    down_chg[diff < 0] = diff[diff < 0]
    up_chg_avg = up_chg.rolling(n).mean()
    down_chg_avg = down_chg.rolling(n).mean().abs()
    rvi = 100 * up_chg_avg / (up_chg_avg + down_chg_avg)
    return rvi

def CCI(df, high_col, low_col, close_col, n=20):
    typical_price = (df[high_col] + df[low_col] + df[close_col]) / 3
    moving_average = typical_price.rolling(n).mean()
    mean_deviation = typical_price.rolling(n).apply(lambda x: np.fabs(x - x.mean()).mean(), raw=True)
    cci = (typical_price - moving_average) / (0.015 * mean_deviation)
    return cci

def MFI(df, high_col, low_col, close_col, volume_col, n=14):
    typical_price = (df[high_col] + df[low_col] + df[close_col]) / 3
    raw_money_flow = typical_price * df[volume_col]
    flow_direction = np.where(typical_price > typical_price.shift(1), 1, -1)
    money_flow_ratio = raw_money_flow.rolling(n).sum() / (df[volume_col].rolling(n).sum() * flow_direction)
    mfi = 100 - (100 / (1 + money_flow_ratio))
    return mfi

def PSAR(df, high_col, low_col, close_col, af=0.02, af_max=0.2):
    psar = df[close_col].copy()
    psar_bull = df[low_col].copy()
    psar_bear = df[high_col].copy()
    psar_dir = 1
    af_current = af
    for i in range(2, len(df)):
        if psar_dir == 1:
            psar[i] = psar[i-1] + af_current * (psar_bull[i-1] - psar[i-1])
        else:
            psar[i] = psar[i-1] + af_current * (psar_bear[i-1] - psar[i-1])
        if psar_dir == 1 and df[high_col][i] > psar[i]:
            psar_dir = -1
            psar[i] = psar_bear[i-1]
            psar_bull[i] = psar[i]
            af_current = af
        elif psar_dir == -1 and df[low_col][i] < psar[i]:
            psar_dir = 1
            psar[i] = psar_bull[i-1]
            psar_bear[i] = psar[i]
            af_current = af
        else:
            if psar_dir == 1:
                if df[low_col][i] < psar_bull[i-1]:
                    psar_bull[i] = df[low_col][i]
                    af_current = min(af_current + af, af_max)
                else:
                    psar_bull[i] = psar_bull[i-1]
            else:
                if df[high_col][i] > psar_bear[i-1]:
                    psar_bear[i] = df[high_col][i]
                    af_current = min(af_current + af, af_max)
                else:
                    psar_bear[i] = psar_bear[i-1]
    return psar

def Ichimoku(df, high_col, low_col, close_col ,suffix='_ichimoku'):
    nine_period_high = df[high_col].rolling(window=9).max()
    nine_period_low = df[low_col].rolling(window=9).min()
    tenkan_sen = (nine_period_high + nine_period_low) / 2

    period26_high = df[high_col].rolling(window=26).max()
    period26_low = df[low_col].rolling(window=26).min()
    kijun_sen = (period26_high + period26_low) / 2

    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)

    period52_high = df[high_col].rolling(window=52).max()
    period52_low = df[low_col].rolling(window=52).min()
    senkou_span_b = ((period52_high + period52_low) / 2).shift(26)

    chikou_span = df[close_col].shift(-26)

    ichimoku = pd.concat([tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b, chikou_span], axis=1, keys=[f'Tenkan_Sen_{suffix}', f'Kijun_Sen_{suffix}', f'Senkou_Span_A_{suffix}', f'Senkou_Span_B_{suffix}', f'Chikou_Span_{suffix}'])
    return ichimoku

def ForceIndex(df, close_col, volume_col, n=13):
    force_index = df[close_col].diff(1) * df[volume_col]
    force_index = force_index.rolling(n).sum()
    return force_index

def UltimateOscillator(df, high_col, low_col, close_col, short_period=7, medium_period=14, long_period=28, weight1=4.0, weight2=2.0, weight3=1.0):
    min_low_or_prev_close = pd.concat([df[low_col], df[close_col].shift(1)], axis=1).min(axis=1)
    max_high_or_prev_close = pd.concat([df[high_col], df[close_col].shift(1)], axis=1).max(axis=1)
    buying_pressure = df[close_col] - min_low_or_prev_close
    true_range = max_high_or_prev_close - min_low_or_prev_close
    avg1 = buying_pressure.rolling(window=short_period).sum() / true_range.rolling(window=short_period).sum()
    avg2 = buying_pressure.rolling(window=medium_period).sum() / true_range.rolling(window=medium_period).sum()
    avg3 = buying_pressure.rolling(window=long_period).sum() / true_range.rolling(window=long_period).sum()
    ultimate_oscillator = 100 * ((weight1 * avg1) + (weight2 * avg2) + (weight3 * avg3)) / (weight1 + weight2 + weight3)
    return ultimate_oscillator

def KeltnerChannels(df, high_col, low_col, close_col, n=20, atr_multiplier=2):
    mid = df[close_col].rolling(n).mean()
    atr = ATR(df, high_col, low_col, close_col, n)
    upper = mid + (atr_multiplier * atr)
    lower = mid - (atr_multiplier * atr)
    return upper, mid, lower
def DonchianChannels(df, high_col, low_col, close_col, n=20):
    upper = df[high_col].rolling(window=n).max()
    lower = df[low_col].rolling(window=n).min()
    middle = (upper + lower) / 2
    return upper, middle, lower
def CMF(df, high_col, low_col, close_col, volume_col, n=20):
    mfm = ((df[close_col] - df[low_col]) - (df[high_col] - df[close_col])) / (df[high_col] - df[low_col])
    mfv = mfm * df[volume_col]
    cmf = mfv.rolling(n).sum() / df[volume_col].rolling(n).sum()
    return cmf
def PSARmod(df, high_col, low_col, close_col, af_start=0.02, af_increment=0.02, af_max=0.2):
    psar = df[close_col].copy()
    psar_direction = pd.Series([1] * len(df), index=df.index)
    psar_extreme_point = df[low_col][0]
    psar_af = af_start
    for i in range(1, len(df)):
        prev_psar = psar[i - 1]
        curr_close = df[close_col][i]
        curr_high = df[high_col][i]
        curr_low = df[low_col][i]
        if psar_direction[i - 1] == 1:
            if curr_low < psar_extreme_point:
                psar_direction[i] = -1
                psar[i] = psar_extreme_point
                psar_af = af_start
                psar_extreme_point = curr_high
            else:
                psar_direction[i] = 1
                psar[i] = prev_psar + psar_af * (psar_extreme_point - prev_psar)
                if curr_high > psar_extreme_point:
                    psar_extreme_point = curr_high
                    psar_af = min(psar_af + af_increment, af_max)
        else:
            if curr_high > psar_extreme_point:
                psar_direction[i] = 1
                psar[i] = psar_extreme_point
                psar_af = af_start
                psar_extreme_point = curr_low
            else:
                psar_direction[i] = -1
                psar[i] = prev_psar + psar_af * (psar_extreme_point - prev_psar)
                if curr_low < psar_extreme_point:
                    psar_extreme_point = curr_low
                    psar_af = min(psar_af + af_increment, af_max)
    return psar


def FibonacciRetracements(df, high_col, low_col, trend='uptrend', levels=[0, 23.6, 38.2, 50, 61.8, 100]):
    if trend == 'uptrend':
        start = df[low_col].idxmin()
        end = df[high_col].idxmax()
        trend_data = df.loc[start:end, :]
    elif trend == 'downtrend':
        start = df[high_col].idxmax()
        end = df[low_col].idxmin()
        trend_data = df.loc[start:end, :]
    else:
        raise ValueError("Invalid trend parameter: must be 'uptrend' or 'downtrend'")
    
    trend_high = trend_data[high_col].max()
    trend_low = trend_data[low_col].min()
    price_range = trend_high - trend_low
    
    retracement_series = pd.Series(np.nan, index=df.index, name='Fibonacci_Retracements')
    
    for level in levels:
        retracement_level = trend_high - (price_range * level / 100)
        mask = (df.index >= start) & (df.index <= end)
        retracement_series[mask] = retracement_level
    
    return retracement_series


def PivotPoints(df, high_col, low_col, close_col):
    pivot = (df[high_col] + df[low_col] + df[close_col]) / 3
    s1 = (pivot * 2) - df[high_col]
    s2 = pivot - (df[high_col] - df[low_col])
    r1 = (pivot * 2) - df[low_col]
    r2 = pivot + (df[high_col] - df[low_col])
    return pivot, s1, s2, r1, r2


def VWAP(df, high_col, low_col, close_col, volume_col):
    typical_price = (df[high_col] + df[low_col] + df[close_col]) / 3
    vwap = (typical_price * df[volume_col]).cumsum() / df[volume_col].cumsum()
    return vwap

def ADL(df, high_col, low_col, close_col, volume_col):
    money_flow_multiplier = ((df[close_col] - df[low_col]) - (df[high_col] - df[close_col])) / (df[high_col] - df[low_col])
    money_flow_volume = money_flow_multiplier * df[volume_col]
    adl = money_flow_volume.cumsum()
    return adl

def calculate_technical_indicators(df):
    # Moving Averages
    df['SMA_10'] = SMA(df, 'close', 10)
    df['EMA_20'] = EMA(df, 'close', 20)
    df['WMA_10'] = WMA(df, 'close', 10)
    df['SMA_20'], df['Upper_BB'], df['Lower_BB'] = Bollinger_Bands(df, 'close', 20, 2)
    df['MACD'], df['Signal'] = MACD(df, 'close', 12, 26, 9)

    # Momentum Indicators
    df['Stochastic_Oscillator'] = Stochastic_Oscillator(df, 'high', 'low', 'close', 14)
    df['ROC_12'] = ROC(df, 'close', 12)
    df['ForceIndex_13'] = ForceIndex(df, 'close', 'volume', 13)
    df['Ultimate_Oscillator'] = UltimateOscillator(df, 'high', 'low', 'close')
    df['RVI_14'] = RVI(df, 'close', 'high', 'low', 14)
    df['delta'], df['avg_gain'], df['avg_loss'] ,  df['RSI_14']= RSI(df , n = 14)
    
    # Trend Indicators
    df['PSAR'] = PSAR(df, 'high', 'low', 'close')
    #df['Ichimoku'] = Ichimoku(df, 'high', 'low','close')
    
    ichimoku = Ichimoku(df, 'high', 'low', 'close')
    df = df.join(ichimoku)  # Join the Ichimoku DataFrame with the original DataFrame
    
    df['Donchian_High'], df['Donchian_Middle'], df['Donchian_Low'] = DonchianChannels(df, 'high', 'low', 'close')
    df['Fibonacci_Retracements'] = FibonacciRetracements(df, 'high', 'low')
    df['Pivot_Points'], df['Pivot_Support_1'], df['Pivot_Support_2'], df['Pivot_Resistance_1'], df['Pivot_Resistance_2'] = PivotPoints(df, 'high', 'low', 'close')
    df['Keltner_Upper'], df['Keltner_Middle'], df['Keltner_Lower'] = KeltnerChannels(df, 'high', 'low', 'close', 20)

    # Volume Indicators
    df['ADL'] = ADL(df, 'high', 'low', 'close', 'volume')
    df['MFI_14'] = MFI(df, 'high', 'low', 'close', 'volume', 14)
    df['VWAP'] = VWAP(df, 'high', 'low', 'close', 'volume')
    df['CMF'] = CMF(df, 'high', 'low', 'close', 'volume', 20)

    # Volatility Indicators
    df['ATR'] = ATR(df, 'high', 'low', 'close', 14)
    df['CCI_20'] = CCI(df, 'high', 'low', 'close', 20)
    df['OBV'] = OBV(df, 'close', 'volume')

    return df


def get_num_days_since_update():
    from datetime import datetime
    df = pd.read_csv('./../../Database/Futures_um/klines/Full_Data_2klines.csv')
    last_entry_dt = (df['open_time'].iloc[-1])

    last_entry_datetime = datetime.strptime(last_entry_dt, '%Y-%m-%d %H:%M:%S')

    import datetime
    base = datetime.datetime.today()
    num_days_since_last_entry = str(base -  last_entry_datetime).split(' ')[0]
    return num_days_since_last_entry

def get_date_list():

    num_days_since_last_entry = get_num_days_since_update()
    
    import datetime
    base = datetime.datetime.today()
    date_list = [base - datetime.timedelta(days=x) for x in range(int(num_days_since_last_entry)+1)]
    return date_list


def quick_date_sanity_check():
    df['open_time'] = pd.to_datetime(df['open_time'])
    df.sort_values('open_time',inplace=True)

# Calclulate Technical Indicators for each period of time
def calculate_technical_per_period():
    my_dict = {'df_min':pd.DataFrame,'df_5min':pd.DataFrame(),'df_10min':pd.DataFrame(),'df_15min':pd.DataFrame(),'df_30min':pd.DataFrame(),'df_hour':pd.DataFrame(),'df_2hour':pd.DataFrame(),'df_4hour':pd.DataFrame(),'df_8hour':pd.DataFrame(),'df_12hour':pd.DataFrame(),'df_day':pd.DataFrame(),'df_week':pd.DataFrame()}
    li = [df_min,df_5min,df_10min,df_15min,df_30min,df_hour,df_2hour,df_4hour,df_8hour,df_12hour,df_day,df_week]
    for i in range(len(li)):
        my_dict[list(my_dict.keys())[i]] =  calculate_technical_indicators(li[i])
    return my_dict

# Rename columns
def rename_cols():
    for i in ['df_min' , 'df_5min', 'df_10min' , 'df_15min' , 'df_30min' , 'df_hour' , 'df_2hour' , 'df_4hour', 'df_8hour','df_12hour','df_day','df_week']:
        period = i.split('_')[-1]
        rename_dict1 = {'Chikou_Span__ichimoku': f'Chikou_Span__ichimoku_{period}',
        'Senkou_Span_A__ichimoku': f'Senkou_Span_A__ichimoku_{period}',
        'Fibonacci_Retracements': f'Fibonacci_Retracements_{period}',
        'Donchian_Low': f'Donchian_Low_{period}',
        'Pivot_Resistance_2': f'Pivot_Resistance_2_{period}',
        'quote_volume': f'quote_volume_{period}',
        'OBV': f'OBV_{period}',
        'ADL': f'ADL_{period}',
        'Keltner_Lower': f'Keltner_Lower_{period}',
        'VWAP': f'VWAP_{period}',
        'Upper_BB': f'Upper_BB_{period}',
        'Pivot_Support_2': f'Pivot_Support_2_{period}',
        'ATR': f'ATR_{period}',
        'WMA_10': f'WMA_10_{period}',
        'Stochastic_Oscillator': f'Stochastic_Oscillator_{period}',
        'SMA_10': f'SMA_10_{period}',
        'Pivot_Points': f'Pivot_Points_{period}',
        'Donchian_Middle': f'Donchian_Middle_{period}',
        'CCI_20': f'CCI_20_{period}',
        'SMA_20': f'SMA_20_{period}',
        'Pivot_Resistance_1': f'Pivot_Resistance_1_{period}',
        'MACD': f'MACD_{period}',
        'Kijun_Sen__ichimoku': f'Kijun_Sen__ichimoku_{period}',
        'PSAR': f'PSAR_{period}',
        'Keltner_Middle': f'Keltner_Middle_{period}',
        'Donchian_High': f'Donchian_High_{period}',
        'Senkou_Span_B__ichimoku': f'Senkou_Span_B__ichimoku_{period}',
        'Lower_BB': f'Lower_BB_{period}',
        'Tenkan_Sen__ichimoku': f'Tenkan_Sen__ichimoku_{period}',
        'ROC_12': f'ROC_12_{period}',
        'Ultimate_Oscillator': f'Ultimate_Oscillator_{period}',
        'CMF': f'CMF_{period}',
        'Pivot_Support_1': f'Pivot_Support_1_{period}',
        'ForceIndex_13': f'ForceIndex_13_{period}',
        'MFI_14': f'MFI_14_{period}',
        'Signal': f'Signal_{period}',
        'RVI_14': f'RVI_14_{period}',
        'Keltner_Upper': f'Keltner_Upper_{period}',
        'index': f'index_{period}',
        'EMA_20': f'EMA_20_{period}'}
        (my_dict[i]).rename(columns=rename_dict1 , inplace=True)
        (my_dict[i]).rename(columns={'open':f'open_{period}'	,'high': f'high_{period}', 	'low':f'low_{period}'	, 'close':f'close_{period}',	'volume':f'volume_{period}'	,'close_time':f'close_time_{period}','quota_volume':f'quote_volume_{period}'	,'count':f'count_{period}',	'taker_buy_volume':f'taker_buy_volume_{period}'	,'taker_buy_quote_volume':f'taker_buy_quote_volume_{period}','ignore':f'ignore_{period}','RSI_14':f'RSI_14_{period}','RS':f'RS_{period}','upPrices':f'upPrices_{period}','down_Prices':f'downPrices_{period}','delta':f'delta_{period}','avg_gain':f'avg_gain_{period}','avg_loss':f'avg_loss_{period}'},inplace=True)
    return my_dict


def save_csv_for_each_period(init=False):
 

    for i in ['df_min' , 'df_5min', 'df_10min' , 'df_15min' , 'df_30min' , 'df_hour' , 'df_2hour' , 'df_4hour', 'df_8hour','df_12hour','df_day','df_week']:
        
        '''try:
            my_dict[i].drop(columns=['index'],inplace=True)
            my_dict[i].drop(columns=[f'index_{i[3:]}'],inplace=True)

        except Exception as e:
            print(e , i)'''
            
        if type(my_dict[i].index.tolist()[0]) == int:
            my_dict[i].reset_index(inplace=True,drop=True)

        else:
            my_dict[i].reset_index(inplace=True)

        # Append or save as a new csv
        if init:
            my_dict[i].to_csv(f'./../../Storage/TechnicalIndicators/{i}/{i[3:]}_data.csv',index=False)

        else:
            pruv = pd.read_csv(f'./../../Storage/TechnicalIndicators/{i}/{i[3:]}_data.csv')

            if my_dict[i]['open_time'].iloc[-1] > pd.to_datetime(pruv['open_time'].iloc[-1]):
                import datetime
                dt = date_list[-2]
                dt_midnight = datetime.datetime(dt.year, dt.month, dt.day, 0, 0, 0)
                df_pruv = my_dict[i].loc[my_dict[i]['open_time']>=dt_midnight]
                df_pruv.to_csv(f'./../../Storage/TechnicalIndicators/{i}/{i[3:]}_data.csv',mode='a',index=False,header=False)
            else:
                print(f"Nothing was apendd for {i}")

date_list = get_date_list()
df = pd.read_csv('./../../Database/Futures_um/klines/Full_Data_2klines.csv')
quick_date_sanity_check()

df_min = df.resample('min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_5min = df.resample('5min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_10min = df.resample('10min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_15min = df.resample('15min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_30min = df.resample('30min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_hour = df.resample('h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_2hour = df.resample('2h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_4hour = df.resample('4h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_8hour = df.resample('8h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_12hour = df.resample('12h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_day = df.resample('d' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_week = df.resample('w' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})

my_dict = calculate_technical_per_period()

my_dict = rename_cols()

save_csv_for_each_period(init=False)



Nothing was apendd for df_week


In [51]:
df = pd.read_csv('./../../Database/Futures_um/klines/Full_Data_klines.csv')


In [57]:
df.sort_values('open_time')

,open_time,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
0,2021-01-12 07:00:00,36060.5,36060.5,36060.5,36060.5,0.469,1610434859999,1.691237e+04,1,0.469,16912.3745,0.0
1,2021-01-12 07:01:00,36042.0,36042.0,36042.0,36042.0,0.001,1610434919999,3.604200e+01,1,0.001,36.0420,0.0
2,2021-01-12 07:02:00,36042.0,36042.0,35856.5,35856.5,0.906,1610434979999,3.254637e+04,4,0.905,32510.4011,0.0
3,2021-01-12 07:03:00,35829.1,35843.3,35824.0,35843.3,4.397,1610435039999,1.575472e+05,12,2.849,102087.7125,0.0
4,2021-01-12 07:04:00,35834.3,35848.1,35807.9,35815.6,4.650,1610435099999,1.666101e+05,9,3.636,130273.7212,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1168276,2023-04-12 23:55:00,29883.4,29883.4,29883.3,29883.3,1.319,1681343759999,3.941610e+04,56,0.261,7799.5674,0.0
1168277,2023-04-12 23:56:00,29883.3,29887.4,29883.3,29883.5,33.548,1681343819999,1.002554e+06,233,25.875,773238.7211,0.0
1168278,2023-04-12 23:57:00,29883.5,29887.5,29883.5,29887.4,3.811,1681343879999,1.138913e+05,102,3.461,103431.5111,0.0
1168279,2023-04-12 23:58:00,29887.4,29887.5,29887.4,29887.4,0.801,1681343939999,2.393983e+04,38,0.216,6455.7000,0.0


In [6]:
df = pd.read_csv('./../../Database/Futures_um/klines/Full_Data_2klines.csv')
df = df.sort_values('open_time').reset_index(drop=True)


In [58]:
quick_date_sanity_check()

In [10]:
df_min

,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,ignore
open_time,,,,,,,,,,,
2021-01-12 07:00:00,36060.5,36060.5,36060.5,36060.5,0.469,1.610435e+12,16912.3745,1,0.469,16912.3745,0.0
2021-01-12 07:01:00,36042.0,36042.0,36042.0,36042.0,0.001,1.610435e+12,36.0420,1,0.001,36.0420,0.0
2021-01-12 07:02:00,36042.0,36042.0,35856.5,35856.5,0.906,1.610435e+12,32546.3687,4,0.905,32510.4011,0.0
2021-01-12 07:03:00,35829.1,35843.3,35824.0,35843.3,4.397,1.610435e+12,157547.2001,12,2.849,102087.7125,0.0
2021-01-12 07:04:00,35834.3,35848.1,35807.9,35815.6,4.650,1.610435e+12,166610.1238,9,3.636,130273.7212,0.0
...,...,...,...,...,...,...,...,...,...,...,...
2023-04-11 23:55:00,30244.9,30245.0,30232.0,30232.1,19.360,1.681257e+12,585371.0273,233,2.281,68972.0213,0.0
2023-04-11 23:56:00,30232.0,30239.3,30228.7,30239.3,13.031,1.681257e+12,393944.5501,173,9.860,298078.3649,0.0
2023-04-11 23:57:00,30239.3,30239.3,30224.9,30224.9,20.222,1.681257e+12,611305.3757,215,1.291,39031.4307,0.0


In [59]:
df_min = df.resample('min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_5min = df.resample('5min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_10min = df.resample('10min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_15min = df.resample('15min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_30min = df.resample('30min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_hour = df.resample('h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_2hour = df.resample('2h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_4hour = df.resample('4h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_8hour = df.resample('8h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_12hour = df.resample('12h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_day = df.resample('d' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_week = df.resample('w' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})


In [199]:
'''import pandas as pd
df = pd.read_csv('./../../Database/Futures_um/klines/Full_Data_2klines.csv')

df['open_time'] = pd.to_datetime(df['open_time'])

# Get the range of dates and set the frequency to be 1 minute
date_range = pd.date_range(start=df['open_time'].min(), end=df['open_time'].max(), freq='min')

# Find the missing dates by comparing the date range to the unique values in the datetime column
missing_dates = date_range[~date_range.isin(df['open_time'])]
print(missing_dates)'''

DatetimeIndex([], dtype='datetime64[ns]', freq='T')


In [11]:
def RSI(df, n=14):
    delta = df['close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(n).mean()
    avg_loss = loss.rolling(n).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return delta, avg_gain, avg_loss , rsi
def SMA(df, column, period):
    return df[column].rolling(period).mean()
def EMA(df, column, period):
    try:
        return df[column].ewm(span=period, adjust=False).mean()
    except KeyError:
        return df.ewm(span=period, adjust=False).mean()
def WMA(df, column, period):
    weights = np.arange(1, period+1)
    wma = df[column].rolling(period).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
    return wma
def Bollinger_Bands(df, column, period, std):
    sma = SMA(df, column, period)
    std_dev = df[column].rolling(period).std()
    upper_band = sma + (std * std_dev)
    lower_band = sma - (std * std_dev)
    return sma, upper_band, lower_band
def MACD(df, column, fast_period, slow_period, signal_period):
    ema_fast = EMA(df, column, fast_period)
    ema_slow = EMA(df, column, slow_period)
    macd_line = ema_fast - ema_slow
    signal_line = EMA(macd_line, column, signal_period)
    return macd_line, signal_line

def Stochastic_Oscillator(df, high_col, low_col, close_col, n=14):
    highest_high = df[high_col].rolling(n).max()
    lowest_low = df[low_col].rolling(n).min()
    k = ((df[close_col] - lowest_low) / (highest_high - lowest_low)) * 100
    return k

def ATR(df, high_col, low_col, close_col, n=14):
    tr1 = abs(df[high_col] - df[low_col])
    tr2 = abs(df[high_col] - df[close_col].shift())
    tr3 = abs(df[low_col] - df[close_col].shift())
    true_range = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = true_range.rolling(n).mean()
    return atr

def ROC(df, column, n=12):
    roc = 100 * (df[column] / df[column].shift(n) - 1)
    return roc


def OBV(df, close_col, volume_col):
    change_in_volume = np.where(df[close_col] > df[close_col].shift(1), df[volume_col], np.where(df[close_col] < df[close_col].shift(1), -df[volume_col], 0))
    obv = change_in_volume.cumsum()
    return obv

def RVI(df, close_col, high_col, low_col, n=14):
    diff = df[close_col].diff(1)
    up_chg = 0 * diff
    down_chg = 0 * diff
    up_chg[diff > 0] = diff[diff > 0]
    down_chg[diff < 0] = diff[diff < 0]
    up_chg_avg = up_chg.rolling(n).mean()
    down_chg_avg = down_chg.rolling(n).mean().abs()
    rvi = 100 * up_chg_avg / (up_chg_avg + down_chg_avg)
    return rvi

def CCI(df, high_col, low_col, close_col, n=20):
    typical_price = (df[high_col] + df[low_col] + df[close_col]) / 3
    moving_average = typical_price.rolling(n).mean()
    mean_deviation = typical_price.rolling(n).apply(lambda x: np.fabs(x - x.mean()).mean(), raw=True)
    cci = (typical_price - moving_average) / (0.015 * mean_deviation)
    return cci

def MFI(df, high_col, low_col, close_col, volume_col, n=14):
    typical_price = (df[high_col] + df[low_col] + df[close_col]) / 3
    raw_money_flow = typical_price * df[volume_col]
    flow_direction = np.where(typical_price > typical_price.shift(1), 1, -1)
    money_flow_ratio = raw_money_flow.rolling(n).sum() / (df[volume_col].rolling(n).sum() * flow_direction)
    mfi = 100 - (100 / (1 + money_flow_ratio))
    return mfi

def PSAR(df, high_col, low_col, close_col, af=0.02, af_max=0.2):
    psar = df[close_col].copy()
    psar_bull = df[low_col].copy()
    psar_bear = df[high_col].copy()
    psar_dir = 1
    af_current = af
    for i in range(2, len(df)):
        if psar_dir == 1:
            psar[i] = psar[i-1] + af_current * (psar_bull[i-1] - psar[i-1])
        else:
            psar[i] = psar[i-1] + af_current * (psar_bear[i-1] - psar[i-1])
        if psar_dir == 1 and df[high_col][i] > psar[i]:
            psar_dir = -1
            psar[i] = psar_bear[i-1]
            psar_bull[i] = psar[i]
            af_current = af
        elif psar_dir == -1 and df[low_col][i] < psar[i]:
            psar_dir = 1
            psar[i] = psar_bull[i-1]
            psar_bear[i] = psar[i]
            af_current = af
        else:
            if psar_dir == 1:
                if df[low_col][i] < psar_bull[i-1]:
                    psar_bull[i] = df[low_col][i]
                    af_current = min(af_current + af, af_max)
                else:
                    psar_bull[i] = psar_bull[i-1]
            else:
                if df[high_col][i] > psar_bear[i-1]:
                    psar_bear[i] = df[high_col][i]
                    af_current = min(af_current + af, af_max)
                else:
                    psar_bear[i] = psar_bear[i-1]
    return psar

def Ichimoku(df, high_col, low_col, close_col ,suffix='_ichimoku'):
    nine_period_high = df[high_col].rolling(window=9).max()
    nine_period_low = df[low_col].rolling(window=9).min()
    tenkan_sen = (nine_period_high + nine_period_low) / 2

    period26_high = df[high_col].rolling(window=26).max()
    period26_low = df[low_col].rolling(window=26).min()
    kijun_sen = (period26_high + period26_low) / 2

    senkou_span_a = ((tenkan_sen + kijun_sen) / 2).shift(26)

    period52_high = df[high_col].rolling(window=52).max()
    period52_low = df[low_col].rolling(window=52).min()
    senkou_span_b = ((period52_high + period52_low) / 2).shift(26)

    chikou_span = df[close_col].shift(-26)

    ichimoku = pd.concat([tenkan_sen, kijun_sen, senkou_span_a, senkou_span_b, chikou_span], axis=1, keys=[f'Tenkan_Sen_{suffix}', f'Kijun_Sen_{suffix}', f'Senkou_Span_A_{suffix}', f'Senkou_Span_B_{suffix}', f'Chikou_Span_{suffix}'])
    return ichimoku

def ForceIndex(df, close_col, volume_col, n=13):
    force_index = df[close_col].diff(1) * df[volume_col]
    force_index = force_index.rolling(n).sum()
    return force_index

def UltimateOscillator(df, high_col, low_col, close_col, short_period=7, medium_period=14, long_period=28, weight1=4.0, weight2=2.0, weight3=1.0):
    min_low_or_prev_close = pd.concat([df[low_col], df[close_col].shift(1)], axis=1).min(axis=1)
    max_high_or_prev_close = pd.concat([df[high_col], df[close_col].shift(1)], axis=1).max(axis=1)
    buying_pressure = df[close_col] - min_low_or_prev_close
    true_range = max_high_or_prev_close - min_low_or_prev_close
    avg1 = buying_pressure.rolling(window=short_period).sum() / true_range.rolling(window=short_period).sum()
    avg2 = buying_pressure.rolling(window=medium_period).sum() / true_range.rolling(window=medium_period).sum()
    avg3 = buying_pressure.rolling(window=long_period).sum() / true_range.rolling(window=long_period).sum()
    ultimate_oscillator = 100 * ((weight1 * avg1) + (weight2 * avg2) + (weight3 * avg3)) / (weight1 + weight2 + weight3)
    return ultimate_oscillator

def KeltnerChannels(df, high_col, low_col, close_col, n=20, atr_multiplier=2):
    mid = df[close_col].rolling(n).mean()
    atr = ATR(df, high_col, low_col, close_col, n)
    upper = mid + (atr_multiplier * atr)
    lower = mid - (atr_multiplier * atr)
    return upper, mid, lower
def DonchianChannels(df, high_col, low_col, close_col, n=20):
    upper = df[high_col].rolling(window=n).max()
    lower = df[low_col].rolling(window=n).min()
    middle = (upper + lower) / 2
    return upper, middle, lower
def CMF(df, high_col, low_col, close_col, volume_col, n=20):
    mfm = ((df[close_col] - df[low_col]) - (df[high_col] - df[close_col])) / (df[high_col] - df[low_col])
    mfv = mfm * df[volume_col]
    cmf = mfv.rolling(n).sum() / df[volume_col].rolling(n).sum()
    return cmf
def PSARmod(df, high_col, low_col, close_col, af_start=0.02, af_increment=0.02, af_max=0.2):
    psar = df[close_col].copy()
    psar_direction = pd.Series([1] * len(df), index=df.index)
    psar_extreme_point = df[low_col][0]
    psar_af = af_start
    for i in range(1, len(df)):
        prev_psar = psar[i - 1]
        curr_close = df[close_col][i]
        curr_high = df[high_col][i]
        curr_low = df[low_col][i]
        if psar_direction[i - 1] == 1:
            if curr_low < psar_extreme_point:
                psar_direction[i] = -1
                psar[i] = psar_extreme_point
                psar_af = af_start
                psar_extreme_point = curr_high
            else:
                psar_direction[i] = 1
                psar[i] = prev_psar + psar_af * (psar_extreme_point - prev_psar)
                if curr_high > psar_extreme_point:
                    psar_extreme_point = curr_high
                    psar_af = min(psar_af + af_increment, af_max)
        else:
            if curr_high > psar_extreme_point:
                psar_direction[i] = 1
                psar[i] = psar_extreme_point
                psar_af = af_start
                psar_extreme_point = curr_low
            else:
                psar_direction[i] = -1
                psar[i] = prev_psar + psar_af * (psar_extreme_point - prev_psar)
                if curr_low < psar_extreme_point:
                    psar_extreme_point = curr_low
                    psar_af = min(psar_af + af_increment, af_max)
    return psar


def FibonacciRetracements(df, high_col, low_col, trend='uptrend', levels=[0, 23.6, 38.2, 50, 61.8, 100]):
    if trend == 'uptrend':
        start = df[low_col].idxmin()
        end = df[high_col].idxmax()
        trend_data = df.loc[start:end, :]
    elif trend == 'downtrend':
        start = df[high_col].idxmax()
        end = df[low_col].idxmin()
        trend_data = df.loc[start:end, :]
    else:
        raise ValueError("Invalid trend parameter: must be 'uptrend' or 'downtrend'")
    
    trend_high = trend_data[high_col].max()
    trend_low = trend_data[low_col].min()
    price_range = trend_high - trend_low
    
    retracement_series = pd.Series(np.nan, index=df.index, name='Fibonacci_Retracements')
    
    for level in levels:
        retracement_level = trend_high - (price_range * level / 100)
        mask = (df.index >= start) & (df.index <= end)
        retracement_series[mask] = retracement_level
    
    return retracement_series


def PivotPoints(df, high_col, low_col, close_col):
    pivot = (df[high_col] + df[low_col] + df[close_col]) / 3
    s1 = (pivot * 2) - df[high_col]
    s2 = pivot - (df[high_col] - df[low_col])
    r1 = (pivot * 2) - df[low_col]
    r2 = pivot + (df[high_col] - df[low_col])
    return pivot, s1, s2, r1, r2


def VWAP(df, high_col, low_col, close_col, volume_col):
    typical_price = (df[high_col] + df[low_col] + df[close_col]) / 3
    vwap = (typical_price * df[volume_col]).cumsum() / df[volume_col].cumsum()
    return vwap

def ADL(df, high_col, low_col, close_col, volume_col):
    money_flow_multiplier = ((df[close_col] - df[low_col]) - (df[high_col] - df[close_col])) / (df[high_col] - df[low_col])
    money_flow_volume = money_flow_multiplier * df[volume_col]
    adl = money_flow_volume.cumsum()
    return adl

def calculate_technical_indicators(df):
    # Moving Averages
    df['SMA_10'] = SMA(df, 'close', 10)
    df['EMA_20'] = EMA(df, 'close', 20)
    df['WMA_10'] = WMA(df, 'close', 10)
    df['SMA_20'], df['Upper_BB'], df['Lower_BB'] = Bollinger_Bands(df, 'close', 20, 2)
    df['MACD'], df['Signal'] = MACD(df, 'close', 12, 26, 9)

    # Momentum Indicators
    df['Stochastic_Oscillator'] = Stochastic_Oscillator(df, 'high', 'low', 'close', 14)
    df['ROC_12'] = ROC(df, 'close', 12)
    df['ForceIndex_13'] = ForceIndex(df, 'close', 'volume', 13)
    df['Ultimate_Oscillator'] = UltimateOscillator(df, 'high', 'low', 'close')
    df['RVI_14'] = RVI(df, 'close', 'high', 'low', 14)
    df['delta'], df['avg_gain'], df['avg_loss'] ,  df['RSI_14']= RSI(df , n = 14)
    # Trend Indicators
    df['PSAR'] = PSAR(df, 'high', 'low', 'close')
    #df['Ichimoku'] = Ichimoku(df, 'high', 'low','close')
    
    ichimoku = Ichimoku(df, 'high', 'low', 'close')
    df = df.join(ichimoku)  # Join the Ichimoku DataFrame with the original DataFrame
    
    df['Donchian_High'], df['Donchian_Middle'], df['Donchian_Low'] = DonchianChannels(df, 'high', 'low', 'close')
    df['Fibonacci_Retracements'] = FibonacciRetracements(df, 'high', 'low')
    df['Pivot_Points'], df['Pivot_Support_1'], df['Pivot_Support_2'], df['Pivot_Resistance_1'], df['Pivot_Resistance_2'] = PivotPoints(df, 'high', 'low', 'close')
    df['Keltner_Upper'], df['Keltner_Middle'], df['Keltner_Lower'] = KeltnerChannels(df, 'high', 'low', 'close', 20)

    # Volume Indicators
    df['ADL'] = ADL(df, 'high', 'low', 'close', 'volume')
    df['MFI_14'] = MFI(df, 'high', 'low', 'close', 'volume', 14)
    df['VWAP'] = VWAP(df, 'high', 'low', 'close', 'volume')
    df['CMF'] = CMF(df, 'high', 'low', 'close', 'volume', 20)

    # Volatility Indicators
    df['ATR'] = ATR(df, 'high', 'low', 'close', 14)
    df['CCI_20'] = CCI(df, 'high', 'low', 'close', 20)
    df['OBV'] = OBV(df, 'close', 'volume')

    return df


In [12]:
def get_num_days_since_update():
    from datetime import datetime
    df = pd.read_csv('./../../Database/Futures_um/klines/Full_Data_2klines.csv')
    last_entry_dt = (df['open_time'].iloc[-1])

    last_entry_datetime = datetime.strptime(last_entry_dt, '%Y-%m-%d %H:%M:%S')

    import datetime
    base = datetime.datetime.today()
    num_days_since_last_entry = str(base -  last_entry_datetime).split(' ')[0]
    return num_days_since_last_entry

In [13]:
get_num_days_since_update()

'8'

In [14]:
def get_date_list():

    num_days_since_last_entry = get_num_days_since_update()
    
    import datetime
    base = datetime.datetime.today()
    date_list = [base - datetime.timedelta(days=x) for x in range(int(num_days_since_last_entry)+1)]
    return date_list
date_list = get_date_list()

In [ ]:
'''my_dict[i].to_csv(f'./../../Storage/TechnicalIndicators/{i}/{i[3:]}_data.csv',index=False)
        '''

In [ ]:
'''for i in ['df_min' , 'df_5min', 'df_10min' , 'df_15min' , 'df_30min' , 'df_hour' , 'df_2hour' , 'df_4hour', 'df_8hour', 'df_12hour','df_day','df_week']:
    os.makedirs(f'./../../Storage/TechnicalIndicators/{i}')'''

In [44]:
df = pd.read_csv('./../../Database/Futures_um/klines/Full_Data_2klines.csv')

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1179801 entries, 0 to 1179800
Data columns (total 12 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   open_time               1179801 non-null  datetime64[ns]
 1   open                    1179801 non-null  float64       
 2   high                    1179801 non-null  float64       
 3   low                     1179801 non-null  float64       
 4   close                   1179801 non-null  float64       
 5   volume                  1179801 non-null  float64       
 6   close_time              1179801 non-null  int64         
 7   quote_volume            1179801 non-null  float64       
 8   count                   1179801 non-null  int64         
 9   taker_buy_volume        1179801 non-null  float64       
 10  taker_buy_quote_volume  1179801 non-null  float64       
 11  ignore                  1179801 non-null  float64       
dtypes: datetime64[

In [16]:
def quick_date_sanity_check():
    df['open_time'] = pd.to_datetime(df['open_time'])
    df.sort_values('open_time',inplace=True)
quick_date_sanity_check()

In [41]:
df_week

,open,high,low,close,volume,close_time,quote_volume,count,taker_buy_volume,taker_buy_quote_volume,...,Stochastic_Oscillator,ROC_12,ForceIndex_13,Ultimate_Oscillator,RVI_14,delta,avg_gain,avg_loss,RSI_14,PSAR
open_time,,,,,,,,,,,,,,,,,,,,,
2021-01-17,36382.069501,36416.085827,36347.784234,36381.962384,7892.467,1.610681e+12,2.864242e+08,110811,3789.286,1.374465e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36381.962384
2021-01-24,33918.818165,33948.955962,33888.417143,33918.433115,8401.730,1.611230e+12,2.852408e+08,114048,4239.344,1.440432e+08,...,NaN,NaN,NaN,NaN,NaN,-2463.529269,NaN,NaN,NaN,33918.433115
2021-01-31,32964.609375,32997.457024,32932.065417,32964.416181,6897.267,1.611835e+12,2.276783e+08,103428,4133.063,1.361074e+08,...,NaN,NaN,NaN,NaN,NaN,-954.016935,NaN,NaN,NaN,33917.832796
2021-02-07,36983.382778,37009.313452,36958.376101,36982.953839,6406.055,1.612440e+12,2.378119e+08,89696,2566.399,9.468891e+07,...,NaN,NaN,NaN,NaN,NaN,4018.537659,NaN,NaN,NaN,32997.457024
2021-02-14,46285.988075,46322.451300,46249.680635,46284.899454,8958.217,1.613045e+12,4.146152e+08,128993,2774.692,1.273421e+08,...,NaN,NaN,NaN,NaN,NaN,9301.945615,NaN,NaN,NaN,33077.694152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-19,25502.543879,25520.545863,25485.686071,25503.126984,963349.313,1.678968e+12,2.438096e+10,8689207,483535.475,1.223484e+10,...,99.803898,51.928291,7.662337e+09,66.027313,75.395623,4188.920952,894.925390,292.047221,75.395623,20450.426438
2023-03-26,27861.245079,27873.711597,27848.640823,27861.244236,589947.195,1.679573e+12,1.647724e+10,6216415,292818.998,8.178923e+09,...,99.889038,67.444038,9.177766e+09,70.964032,78.294416,2358.117252,1053.446280,292.047221,78.294416,21160.243158
2023-04-02,28000.827450,28009.183046,27992.321369,28000.844911,413625.064,1.680178e+12,1.155309e+10,4407483,206553.524,5.769465e+09,...,99.926673,66.577614,9.268815e+09,74.998660,80.209755,139.600675,1063.417757,262.378284,80.209755,22234.398108


In [17]:
df_min = df.resample('min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_5min = df.resample('5min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_10min = df.resample('10min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_15min = df.resample('15min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_30min = df.resample('30min' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_hour = df.resample('h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_2hour = df.resample('2h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_4hour = df.resample('4h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_8hour = df.resample('8h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_12hour = df.resample('12h' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_day = df.resample('d' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})
df_week = df.resample('w' ,on="open_time").agg({'open':'mean','high':'mean','low':'mean','close':'mean', 'volume':'sum','close_time':'mean','quote_volume':'sum','count':'sum','taker_buy_volume':'sum','taker_buy_quote_volume':'sum' ,'ignore':'sum'})#,'RSI':'mean','RS':'mean'})


In [20]:
# Calclulate Technical Indicators for each period of time
def calculate_technical_per_period():
    my_dict = {'df_min':pd.DataFrame,'df_5min':pd.DataFrame(),'df_10min':pd.DataFrame(),'df_15min':pd.DataFrame(),'df_30min':pd.DataFrame(),'df_hour':pd.DataFrame(),'df_2hour':pd.DataFrame(),'df_4hour':pd.DataFrame(),'df_8hour':pd.DataFrame(),'df_12hour':pd.DataFrame(),'df_day':pd.DataFrame(),'df_week':pd.DataFrame()}
    li = [df_min,df_5min,df_10min,df_15min,df_30min,df_hour,df_2hour,df_4hour,df_8hour,df_12hour,df_day,df_week]
    for i in range(len(li)):
        my_dict[list(my_dict.keys())[i]] =  calculate_technical_indicators(li[i])
    return my_dict

my_dict = calculate_technical_per_period()

In [21]:
# Rename columns
def rename_cols():
    for i in ['df_min' , 'df_5min', 'df_10min' , 'df_15min' , 'df_30min' , 'df_hour' , 'df_2hour' , 'df_4hour', 'df_8hour','df_12hour','df_day','df_week']:
        period = i.split('_')[-1]
        rename_dict1 = {'Chikou_Span__ichimoku': f'Chikou_Span__ichimoku_{period}',
        'Senkou_Span_A__ichimoku': f'Senkou_Span_A__ichimoku_{period}',
        'Fibonacci_Retracements': f'Fibonacci_Retracements_{period}',
        'Donchian_Low': f'Donchian_Low_{period}',
        'Pivot_Resistance_2': f'Pivot_Resistance_2_{period}',
        'quote_volume': f'quote_volume_{period}',
        'OBV': f'OBV_{period}',
        'ADL': f'ADL_{period}',
        'Keltner_Lower': f'Keltner_Lower_{period}',
        'VWAP': f'VWAP_{period}',
        'Upper_BB': f'Upper_BB_{period}',
        'Pivot_Support_2': f'Pivot_Support_2_{period}',
        'ATR': f'ATR_{period}',
        'WMA_10': f'WMA_10_{period}',
        'Stochastic_Oscillator': f'Stochastic_Oscillator_{period}',
        'SMA_10': f'SMA_10_{period}',
        'Pivot_Points': f'Pivot_Points_{period}',
        'Donchian_Middle': f'Donchian_Middle_{period}',
        'CCI_20': f'CCI_20_{period}',
        'SMA_20': f'SMA_20_{period}',
        'Pivot_Resistance_1': f'Pivot_Resistance_1_{period}',
        'MACD': f'MACD_{period}',
        'Kijun_Sen__ichimoku': f'Kijun_Sen__ichimoku_{period}',
        'PSAR': f'PSAR_{period}',
        'Keltner_Middle': f'Keltner_Middle_{period}',
        'Donchian_High': f'Donchian_High_{period}',
        'Senkou_Span_B__ichimoku': f'Senkou_Span_B__ichimoku_{period}',
        'Lower_BB': f'Lower_BB_{period}',
        'Tenkan_Sen__ichimoku': f'Tenkan_Sen__ichimoku_{period}',
        'ROC_12': f'ROC_12_{period}',
        'Ultimate_Oscillator': f'Ultimate_Oscillator_{period}',
        'CMF': f'CMF_{period}',
        'Pivot_Support_1': f'Pivot_Support_1_{period}',
        'ForceIndex_13': f'ForceIndex_13_{period}',
        'MFI_14': f'MFI_14_{period}',
        'Signal': f'Signal_{period}',
        'RVI_14': f'RVI_14_{period}',
        'Keltner_Upper': f'Keltner_Upper_{period}',
        'index': f'index_{period}',
        'EMA_20': f'EMA_20_{period}'}
        (my_dict[i]).rename(columns=rename_dict1 , inplace=True)
        (my_dict[i]).rename(columns={'open':f'open_{period}'	,'high': f'high_{period}', 	'low':f'low_{period}'	, 'close':f'close_{period}',	'volume':f'volume_{period}'	,'close_time':f'close_time_{period}','quota_volume':f'quote_volume_{period}'	,'count':f'count_{period}',	'taker_buy_volume':f'taker_buy_volume_{period}'	,'taker_buy_quote_volume':f'taker_buy_quote_volume_{period}','ignore':f'ignore_{period}','RSI_14':f'RSI_14_{period}','RS':f'RS_{period}','upPrices':f'upPrices_{period}','down_Prices':f'downPrices_{period}','delta':f'delta_{period}','avg_gain':f'avg_gain_{period}','avg_loss':f'avg_loss_{period}'},inplace=True)
    return my_dict
my_dict = rename_cols()

In [45]:
my_dict['df_week']

,open_week,high_week,low_week,close_week,volume_week,close_time_week,quote_volume_week,count_week,taker_buy_volume_week,taker_buy_quote_volume_week,...,Keltner_Upper_week,Keltner_Middle_week,Keltner_Lower_week,ADL_week,MFI_14_week,VWAP_week,CMF_week,ATR_week,CCI_20_week,OBV_week
open_time,,,,,,,,,,,,,,,,,,,,,
2021-01-17,36382.069501,36416.085827,36347.784234,36381.962384,7892.467,1.610681e+12,2.864242e+08,110811,3789.286,1.374465e+08,...,NaN,NaN,NaN,6.321687,NaN,36381.944148,NaN,NaN,NaN,0.000
2021-01-24,33918.818165,33948.955962,33888.417143,33918.433115,8401.730,1.611230e+12,2.852408e+08,114048,4239.344,1.440432e+08,...,NaN,NaN,NaN,-64.023703,NaN,35111.778149,NaN,NaN,NaN,-8401.730
2021-01-31,32964.609375,32997.457024,32932.065417,32964.416181,6897.267,1.611835e+12,2.276783e+08,103428,4133.063,1.361074e+08,...,NaN,NaN,NaN,-136.810745,NaN,34473.209481,NaN,NaN,NaN,-15298.997
2021-02-07,36983.382778,37009.313452,36958.376101,36982.953839,6406.055,1.612440e+12,2.378119e+08,89696,2566.399,9.468891e+07,...,NaN,NaN,NaN,-360.905422,NaN,35016.544391,NaN,NaN,NaN,-8892.942
2021-02-14,46285.988075,46322.451300,46249.680635,46284.899454,8958.217,1.613045e+12,4.146152e+08,128993,2774.692,1.273421e+08,...,NaN,NaN,NaN,-648.105647,NaN,37634.866512,NaN,NaN,NaN,65.275
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-19,25502.543879,25520.545863,25485.686071,25503.126984,963349.313,1.678968e+12,2.438096e+10,8689207,483535.475,1.223484e+10,...,22172.873451,19953.527145,17734.180840,-267879.123870,99.995400,27090.471945,-0.001367,1192.995105,129.020860,-87408.209
2023-03-26,27861.245079,27873.711597,27848.640823,27861.244236,589947.195,1.679573e+12,1.647724e+10,6216415,292818.998,8.178923e+09,...,22726.196206,20311.799205,17897.402204,-264677.656978,99.995555,27102.057830,-0.001173,1352.184009,156.020666,502538.986
2023-04-02,28000.827450,28009.183046,27992.321369,28000.844911,413625.064,1.680178e+12,1.155309e+10,4407483,206553.524,5.769465e+09,...,22963.999169,20817.812503,18671.625837,-260129.540022,99.995651,27111.431206,0.002820,1332.944789,139.281520,916164.050


In [167]:
def save_csv_for_each_period(init=False):
 

    for i in ['df_min' , 'df_5min', 'df_10min' , 'df_15min' , 'df_30min' , 'df_hour' , 'df_2hour' , 'df_4hour', 'df_8hour','df_12hour','df_day','df_week']:
        
        '''try:
            my_dict[i].drop(columns=['index'],inplace=True)
            my_dict[i].drop(columns=[f'index_{i[3:]}'],inplace=True)

        except Exception as e:
            print(e , i)'''
            
        if type(my_dict[i].index.tolist()[0]) == int:
            my_dict[i].reset_index(inplace=True,drop=True)

        else:
            my_dict[i].reset_index(inplace=True)

        # Append or save as a new csv
        if init:
            my_dict[i].to_csv(f'./../../Storage/TechnicalIndicators/{i}/{i[3:]}_data.csv',index=False)

        else:
            pruv = pd.read_csv(f'./../../Storage/ so I want toTechnicalIndicators/{i}/{i[3:]}_data.csv')

            if my_dict[i]['open_time'].iloc[-1] > pd.to_datetime(pruv['open_time'].iloc[-1]):
                import datetime
                dt = date_list[-2]
                dt_midnight = datetime.datetime(dt.year, dt.month, dt.day, 0, 0, 0)
                df_pruv = my_dict[i].loc[my_dict[i]['open_time']>=dt_midnight]
                df_pruv.to_csv(f'./../../Storage/TechnicalIndicators/{i}/{i[3:]}_data.csv',mode='a',index=False,header=False)
            else:
                print(f"Nothing was apendd for {i}")

save_csv_for_each_period()

In [44]:
df_week[df_week.columns.tolist()[-10:]]

,Stochastic_Oscillator,ROC_12,ForceIndex_13,Ultimate_Oscillator,RVI_14,delta,avg_gain,avg_loss,RSI_14,PSAR
open_time,,,,,,,,,,
2021-01-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36381.962384
2021-01-24,NaN,NaN,NaN,NaN,NaN,-2463.529269,NaN,NaN,NaN,33918.433115
2021-01-31,NaN,NaN,NaN,NaN,NaN,-954.016935,NaN,NaN,NaN,33917.832796
2021-02-07,NaN,NaN,NaN,NaN,NaN,4018.537659,NaN,NaN,NaN,32997.457024
2021-02-14,NaN,NaN,NaN,NaN,NaN,9301.945615,NaN,NaN,NaN,33077.694152
...,...,...,...,...,...,...,...,...,...,...
2023-03-19,99.803898,51.928291,7.662337e+09,66.027313,75.395623,4188.920952,894.925390,292.047221,75.395623,20450.426438
2023-03-26,99.889038,67.444038,9.177766e+09,70.964032,78.294416,2358.117252,1053.446280,292.047221,78.294416,21160.243158
2023-04-02,99.926673,66.577614,9.268815e+09,74.998660,80.209755,139.600675,1063.417757,262.378284,80.209755,22234.398108


In [ ]:
for i in ['df_min' , 'df_5min', 'df_10min' , 'df_15min' , 'df_30min' , 'df_hour' , 'df_2hour' , 'df_4hour', 'df_8hour','df_12hour','df_day','df_week']:
    
    try:
        my_dict[i].drop(columns=['index'],inplace=True)
        my_dict[i].drop(columns=[f'index_{i[3:]}'],inplace=True)

    except Exception as e:
        print(e , i)
        
    if my_dict[i].index.tolist()[0] == 0 :
        my_dict[i].reset_index(inplace=True,drop=True)

    else:
        my_dict[i].reset_index(inplace=True)

    my_dict[i].to_csv(f'./../../Storage/TechnicalIndicators/{i}/{i[3:]}_data.csv',index=False)

"['index'] not found in axis" df_min
"['index'] not found in axis" df_5min
"['index'] not found in axis" df_10min
"['index'] not found in axis" df_15min
"['index'] not found in axis" df_30min
"['index'] not found in axis" df_hour
"['index'] not found in axis" df_2hour
"['index'] not found in axis" df_4hour
"['index'] not found in axis" df_8hour
"['index'] not found in axis" df_12hour
"['index'] not found in axis" df_day
"['index'] not found in axis" df_week


In [31]:
hot_df_week.iloc[-15:]['RSI_14_week']

open_time
2022-09-04    27.051469
2022-09-11    23.743695
2022-09-18    25.792600
2022-11-13    29.122065
2022-11-20    21.122274
2022-11-27    22.764894
2022-12-04    29.857444
2023-02-19    74.087961
2023-02-26    83.793125
2023-03-05    78.975501
2023-03-19    75.395623
2023-03-26    78.294416
2023-04-02    80.209755
2023-04-09    80.905455
2023-04-16    82.016371
Name: RSI_14_week, dtype: float64

# Some quick analysis for RSI strategy


In [61]:
hot_df_week.iloc[-10:][hot_df_week.columns.tolist()[20:]]

,ROC_12_week,ForceIndex_13_week,Ultimate_Oscillator_week,RVI_14_week,delta_week,avg_gain_week,avg_loss_week,RSI_14_week,PSAR_week,Tenkan_Sen__ichimoku_week,...,Keltner_Upper_week,Keltner_Middle_week,Keltner_Lower_week,ADL_week,MFI_14_week,VWAP_week,CMF_week,ATR_week,CCI_20_week,OBV_week
open_time,,,,,,,,,,,,,,,,,,,,,
2022-11-27,-18.179897,-5.479119e+09,24.118565,22.764894,-268.361220,192.550342,653.271038,22.764894,20161.370830,18527.257659,...,21997.978217,20289.701562,18581.424908,-325904.972975,100.005211,28803.624582,-0.006126,852.574205,-170.751258,-4527919.491
2022-12-04,-16.822073,-4.666377e+09,30.408840,29.857444,422.836885,222.752976,523.302100,29.857444,19628.381583,18527.257659,...,21828.983033,20108.821631,18388.660229,-332969.051802,99.994721,28599.052800,-0.005441,752.510047,-131.075329,-4010727.225
2023-02-19,43.431899,5.194340e+09,79.234106,74.087961,1062.358968,613.271527,214.489853,74.087961,18307.545339,20054.737703,...,20886.565613,19189.047997,17491.530381,-278130.935317,99.994892,27351.727170,0.000410,832.145684,135.492150,-522868.864
2023-02-26,42.549574,5.623568e+09,81.986933,83.793125,458.415734,646.015508,124.949302,83.793125,18823.939755,20284.092669,...,21100.519200,19398.155839,17695.792478,-269771.795268,99.995054,27293.787299,0.000348,775.511488,128.369829,94501.531
2023-03-05,34.767522,5.011198e+09,73.399267,78.975501,-926.781349,646.015508,171.979311,78.975501,19436.690917,20369.316885,...,21328.473081,19588.511603,17848.550125,-266957.378244,100.004836,27244.397459,-0.000152,822.611492,89.494154,-330283.457
2023-03-19,51.928291,7.662337e+09,66.027313,75.395623,4188.920952,894.925390,292.047221,75.395623,20450.426438,23413.761176,...,22172.873451,19953.527145,17734.180840,-267879.123870,99.995400,27090.471945,-0.001367,1192.995105,129.020860,-87408.209
2023-03-26,67.444038,9.177766e+09,70.964032,78.294416,2358.117252,1053.446280,292.047221,78.294416,21160.243158,24590.344043,...,22726.196206,20311.799205,17897.402204,-264677.656978,99.995555,27102.057830,-0.001173,1352.184009,156.020666,502538.986
2023-04-02,66.577614,9.268815e+09,74.998660,80.209755,139.600675,1063.417757,262.378284,80.209755,22234.398108,24658.079767,...,22963.999169,20817.812503,18671.625837,-260129.540022,99.995651,27111.431206,0.002820,1332.944789,139.281520,916164.050
2023-04-09,49.921355,9.247541e+09,74.253607,80.905455,52.758343,1067.186210,251.867252,80.905455,23273.859397,24683.141706,...,23415.219392,21389.142104,19363.064816,-256502.833982,99.995713,27117.488358,0.005124,1326.527334,127.539360,1172791.072


In [62]:

hot_df_min = my_dict['df_min'].loc[((my_dict['df_min']['RSI_14_min']>=70) | (my_dict['df_min']['RSI_14_min']<=30) )]
hot_df_5min = my_dict['df_5min'].loc[((my_dict['df_5min']['RSI_14_5min']>=70) | (my_dict['df_5min']['RSI_14_5min']<=30) )]
hot_df_10min = my_dict['df_10min'].loc[((my_dict['df_10min']['RSI_14_10min']>=70) | (my_dict['df_10min']['RSI_14_10min']<=30) )]
hot_df_15min = my_dict['df_15min'].loc[((my_dict['df_15min']['RSI_14_15min']>=70) | (my_dict['df_15min']['RSI_14_15min']<=30) )]
hot_df_30min = my_dict['df_30min'].loc[((my_dict['df_30min']['RSI_14_30min']>=70) | (my_dict['df_30min']['RSI_14_30min']<=30) )]
hot_df_hour = my_dict['df_hour'].loc[((my_dict['df_hour']['RSI_14_hour']>=70) | (my_dict['df_hour']['RSI_14_hour']<=30) )]
hot_df_2hour = my_dict['df_2hour'].loc[((my_dict['df_2hour']['RSI_14_2hour']>=70) | (my_dict['df_2hour']['RSI_14_2hour']<=30) )]
hot_df_4hour = my_dict['df_4hour'].loc[((my_dict['df_4hour']['RSI_14_4hour']>=70) | (my_dict['df_4hour']['RSI_14_4hour']<=30) )]
hot_df_8hour = my_dict['df_8hour'].loc[((my_dict['df_8hour']['RSI_14_8hour']>=70) | (my_dict['df_8hour']['RSI_14_8hour']<=30) )]
hot_df_12hour= my_dict['df_12hour'].loc[((my_dict['df_12hour']['RSI_14_12hour']>=70) | (my_dict['df_12hour']['RSI_14_12hour']<=30) )]
hot_df_day = my_dict['df_day'].loc[((my_dict['df_day']['RSI_14_day']>=70) | (my_dict['df_day']['RSI_14_day']<=30) )]
hot_df_week = my_dict['df_week'].loc[((my_dict['df_week']['RSI_14_week']>=70) | (my_dict['df_week']['RSI_14_week']<=30) )]


In [63]:
# THSI MAY BE A GOOD METRIC TOUSE SMEHOW TO DICERN BETWEENE VENTS INA  DIFFERENC EOF STREAMS

hot_df_min.shape[0] / df_min.shape[0] , \
hot_df_5min.shape[0] / df_5min.shape[0] ,\
hot_df_10min.shape[0] / df_10min.shape[0] ,\
hot_df_15min.shape[0] / df_15min.shape[0] ,\
hot_df_30min.shape[0] / df_30min.shape[0] ,\
hot_df_hour.shape[0] / df_hour.shape[0] ,\
hot_df_2hour.shape[0] / df_2hour.shape[0] ,\
hot_df_4hour.shape[0] / df_4hour.shape[0] ,\
hot_df_8hour.shape[0] / df_8hour.shape[0] ,\
hot_df_12hour.shape[0] / df_12hour.shape[0] ,\
hot_df_day.shape[0] / df_day.shape[0] ,\
hot_df_week.shape[0] / df_week.shape[0]


(0.23484033101487536,
 0.29872146350544077,
 0.3090233707332758,
 0.31896989389247093,
 0.3224095039853785,
 0.34985023099964463,
 0.3751649913696822,
 0.3768527918781726,
 0.39788875355257813,
 0.3751522533495737,
 0.37758830694275275,
 0.4152542372881356)

In [64]:
merge1 = pd.merge(hot_df_5min.reset_index(drop=False) , hot_df_min.reset_index(drop=False) , on = 'open_time' , how='inner')
merge2 = pd.merge(merge1.reset_index(drop=True) , hot_df_10min.reset_index(drop=False) , on = 'open_time' , how='inner')
merge3 = pd.merge(merge2.reset_index(drop=True) , hot_df_15min.reset_index(drop=False) , on = 'open_time' , how='inner')
merge4 = pd.merge(merge3.reset_index(drop=True) , hot_df_30min.reset_index(drop=False) , on = 'open_time' , how='inner')
merge5 = pd.merge(merge4.reset_index(drop=True) , hot_df_hour.reset_index(drop=False) , on = 'open_time' , how='inner')
merge6 = pd.merge(merge5.reset_index(drop=True) , hot_df_4hour.reset_index(drop=False) , on = 'open_time' , how='inner')
merge7 = pd.merge(merge6.reset_index(drop=True) , hot_df_day.reset_index(drop=False) , on = 'open_time' , how='inner')


/var/folders/fn/0t1gngls6x39txrdzml_kv3w0000gn/T/ipykernel_3606/207354091.py:3: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  merge3 = pd.merge(merge2.reset_index(drop=True) , hot_df_15min.reset_index(drop=False) , on = 'open_time' , how='inner')
/var/folders/fn/0t1gngls6x39txrdzml_kv3w0000gn/T/ipykernel_3606/207354091.py:5: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  merge5 = pd.merge(merge4.reset_index(drop=True) , hot_df_hour.reset_index(drop=False) , on = 'open_time' , how='inner')
/var/folders/fn/0t1gngls6x39txrdzml_kv3w0000gn/T/ipykernel_3606/207354091.py:7: FutureWarning: Passing 'suffixes' which cause duplicate columns {'index_x'} in the result is deprecated and will raise a MergeError in a future version.
  merge7 = pd.merge(merge6.reset_index(drop=True) , hot_

In [66]:
interest_cols_1h_level = ['open_time', 'RSI_14_min' ,'RSI_14_5min','RSI_14_10min','RSI_14_15min','RSI_14_30min','RSI_14_hour','volume_min' ,'volume_5min','volume_10min','volume_15min','volume_30min','volume_hour']
interest_cols_4h_level = ['open_time', 'RSI_14_min' ,'RSI_14_5min','RSI_14_10min','RSI_14_15min','RSI_14_30min','RSI_14_hour','RSI_14_4hour','volume_min' ,'volume_5min','volume_10min','volume_15min','volume_30min','volume_hour','volume_4hour']
interest_vol_ratios_4h_level = ['open_time','volume_5min' ,'volume_proj_5min', 'volume_10min' , 'volume_proj_10min' , 'volume_15min', 'volume_proj_15min' , 'volume_30min' ,'volume_proj_30min' , 'volume_hour' , 'volume_proj_hour' ,'volume_4hour' ,   'volume_proj_4hour']
interest_cols_RSI_4h_level = ['open_time', 'RSI_14_min' ,'RSI_14_5min','RSI_14_10min','RSI_14_15min','RSI_14_30min','RSI_14_hour','RSI_14_4hour']#,'volume_min' ,'volume_5min','volume_10min','volume_15min','volume_30min','volume_hour','volume_4hour']
interest_cols_day_level = ['open_time', 'RSI_14_min' ,'RSI_14_5min','RSI_14_10min','RSI_14_15min','RSI_14_30min','RSI_14_hour','RSI_14_4hour','RSI_14_day','volume_min' ,'volume_5min','volume_10min','volume_15min','volume_30min','volume_hour','volume_4hour','volume_day']


In [67]:
my_projecction_dict = {'volume_min':pd.DataFrame() ,'volume_5min':pd.DataFrame(),'volume_10min':pd.DataFrame(),'volume_15min':pd.DataFrame(),'volume_30min':pd.DataFrame(),'volume_hour':pd.DataFrame(),'volume_4hour':pd.DataFrame()}
arr_5min = np.array(merge6['volume_min'].apply(lambda x : x*5))
arr_10min = np.array(merge6['volume_min'].apply(lambda x : x*10))
arr_15min = np.array(merge6['volume_min'].apply(lambda x : x*15))
arr_30min = np.array(merge6['volume_min'].apply(lambda x : x*30))
arr_hour = np.array(merge6['volume_min'].apply(lambda x : x*60))
arr_4hour = np.array(merge6['volume_min'].apply(lambda x : x*240))

In [68]:
merge6['volume_proj_5min'] = arr_5min 

merge6['volume_proj_10min'] = arr_10min 

merge6['volume_proj_15min'] = arr_15min 

merge6['volume_proj_30min'] = arr_30min 

merge6['volume_proj_hour'] = arr_hour 

merge6['volume_proj_4hour'] = arr_4hour 

In [76]:
merge7.iloc[-10:][['open_min']+interest_cols_4h_level]

,open_min,open_time,RSI_14_min,RSI_14_5min,RSI_14_10min,RSI_14_15min,RSI_14_30min,RSI_14_hour,RSI_14_4hour,volume_min,volume_5min,volume_10min,volume_15min,volume_30min,volume_hour,volume_4hour
0,46534.6,2021-02-09,76.178643,79.521334,88.550508,92.434614,94.496130,91.443305,74.240469,3.517,38.173,69.431,78.438,117.400,136.897,295.150
1,41584.0,2022-02-05,73.546722,82.965970,83.049981,84.132960,86.013451,90.656869,85.673737,48.468,213.678,375.835,546.302,914.413,1883.222,3685.816
2,17936.1,2023-01-12,70.454545,88.124054,92.031621,93.981410,97.197430,86.308684,83.841981,141.417,601.116,1337.599,1817.052,2973.145,13180.544,29298.982


In [29]:
my_dict['df_min']

,open_min,high_min,low_min,close_min,volume_min,close_time_min,quote_volume_min,count_min,taker_buy_volume_min,taker_buy_quote_volume_min,...,Keltner_Upper_min,Keltner_Middle_min,Keltner_Lower_min,ADL_min,MFI_14_min,VWAP_min,CMF_min,ATR_min,CCI_20_min,OBV_min
open_time,,,,,,,,,,,,,,,,,,,,,
2021-01-12 07:00:00,36060.5,36060.5,36060.5,36060.5,0.469,1.610435e+12,16912.3745,1,0.469,16912.3745,...,NaN,NaN,NaN,NaN,NaN,36060.500000,NaN,NaN,NaN,0.000
2021-01-12 07:01:00,36042.0,36042.0,36042.0,36042.0,0.001,1.610435e+12,36.0420,1,0.001,36.0420,...,NaN,NaN,NaN,NaN,NaN,36060.460638,NaN,NaN,NaN,-0.001
2021-01-12 07:02:00,36042.0,36042.0,35856.5,35856.5,0.906,1.610435e+12,32546.3687,4,0.905,32510.4011,...,NaN,NaN,NaN,-0.906000,NaN,35966.879724,NaN,NaN,NaN,-0.907
2021-01-12 07:03:00,35829.1,35843.3,35824.0,35843.3,4.397,1.610435e+12,157547.2001,12,2.849,102087.7125,...,NaN,NaN,NaN,3.491000,NaN,35867.855402,NaN,NaN,NaN,-5.304
2021-01-12 07:04:00,35834.3,35848.1,35807.9,35815.6,4.650,1.610435e+12,166610.1238,9,3.636,130273.7212,...,NaN,NaN,NaN,0.622343,NaN,35848.230762,NaN,NaN,NaN,-9.954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-11 23:55:00,30244.9,30245.0,30232.0,30232.1,19.360,1.681257e+12,585371.0273,233,2.281,68972.0213,...,30251.700,30242.230,30232.760,-214278.639976,100.003307,27105.022825,0.078967,2.671429,-118.453758,70117.640
2023-04-11 23:56:00,30232.0,30239.3,30228.7,30239.3,13.031,1.681257e+12,393944.5501,173,9.860,298078.3649,...,30252.415,30242.825,30233.235,-214265.608976,100.003307,27105.023845,0.092446,3.321429,-175.527426,70130.671
2023-04-11 23:57:00,30239.3,30239.3,30224.9,30224.9,20.222,1.681257e+12,611305.3757,215,1.291,39031.4307,...,30249.755,30241.655,30233.555,-214285.830976,100.003307,27105.025423,-0.254168,3.821429,-254.384148,70110.449


In [29]:
techIndicatorRows = my_dict['df_min'].columns.tolist()[12:]

In [30]:
my_dict['df_4hour']

,open_4hour,high_4hour,low_4hour,close_4hour,volume_4hour,close_time_4hour,quote_volume_4hour,count_4hour,taker_buy_volume_4hour,taker_buy_quote_volume_4hour,...,Keltner_Upper_4hour,Keltner_Middle_4hour,Keltner_Lower_4hour,ADL_4hour,MFI_14_4hour,VWAP_4hour,CMF_4hour,ATR_4hour,CCI_20_4hour,OBV_4hour
open_time,,,,,,,,,,,,,,,,,,,,,
2021-01-12 04:00:00,36032.428333,36066.035000,36016.525000,36039.631667,44.903,1.610437e+12,1.613562e+06,342,28.416,1.019288e+06,...,NaN,NaN,NaN,-2.989906,NaN,36040.730556,NaN,NaN,NaN,0.000
2021-01-12 08:00:00,35644.220000,35679.830000,35609.925417,35641.129167,120.431,1.610446e+12,4.294201e+06,1315,58.873,2.096513e+06,...,NaN,NaN,NaN,-15.905815,NaN,35751.477078,NaN,NaN,NaN,-120.431
2021-01-12 12:00:00,34234.229583,34283.076250,34183.186250,34229.842917,153.188,1.610460e+12,5.217053e+06,1764,71.008,2.421972e+06,...,NaN,NaN,NaN,-25.991573,NaN,35020.726073,NaN,NaN,NaN,-273.619
2021-01-12 16:00:00,34766.935417,34802.862083,34735.170833,34770.585833,129.318,1.610474e+12,4.497464e+06,1539,61.377,2.135840e+06,...,NaN,NaN,NaN,-19.995276,NaN,34948.193618,NaN,NaN,NaN,-144.301
2021-01-12 20:00:00,34088.665833,34128.633333,34044.199583,34082.390833,149.083,1.610489e+12,5.078154e+06,1846,83.399,2.843971e+06,...,NaN,NaN,NaN,-34.211208,NaN,34732.627498,NaN,NaN,NaN,-293.384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-11 04:00:00,30078.734167,30085.077917,30072.326250,30078.739583,7048.362,1.681193e+12,2.117464e+08,73910,3614.976,1.085997e+08,...,28683.911750,28391.146125,28098.380500,-244511.235245,99.996547,27111.107698,-0.011576,187.442113,228.700352,-1556200.630
2023-04-11 08:00:00,30091.785000,30095.358333,30087.511250,30091.597083,4814.645,1.681207e+12,1.448742e+08,49750,2015.357,6.063968e+07,...,28791.276104,28499.226854,28207.177604,-244312.084369,99.996556,27111.466455,-0.011725,180.957887,174.612094,-1551385.985
2023-04-11 12:00:00,30145.672083,30154.142917,30137.368750,30146.245417,9687.907,1.681222e+12,2.920988e+08,103571,4919.415,1.483326e+08,...,28887.038417,28603.002333,28318.966250,-243746.567417,99.996572,27112.201254,-0.009207,180.903512,144.807900,-1541698.078


In [40]:
my_dict['df_4hour'].loc[(my_dict['df_4hour'].index)>='2023-04-10 00:00:00']

,open_4hour,high_4hour,low_4hour,close_4hour,volume_4hour,close_time_4hour,quote_volume_4hour,count_4hour,taker_buy_volume_4hour,taker_buy_quote_volume_4hour,...,Keltner_Upper_4hour,Keltner_Middle_4hour,Keltner_Lower_4hour,ADL_4hour,MFI_14_4hour,VWAP_4hour,CMF_4hour,ATR_4hour,CCI_20_4hour,OBV_4hour
open_time,,,,,,,,,,,,,,,,,,,,,
2023-04-10 00:00:00,28329.327917,28332.332500,28325.683333,28329.178750,3649.328,1.681092e+12,1.033885e+08,37229,1903.820,5.393193e+07,...,28131.131937,28003.573771,27876.015604,-242954.913307,99.996436,27107.021225,0.011433,71.882083,258.676242,-1609309.068
2023-04-10 04:00:00,28281.811667,28283.794167,28279.737500,28281.820417,2180.050,1.681106e+12,6.162783e+07,23518,1129.220,3.192062e+07,...,28144.922625,28016.828417,27888.734208,-242896.247212,100.003562,27107.085378,0.009826,74.871369,176.995339,-1611489.118
2023-04-10 08:00:00,28324.153333,28326.531250,28321.972917,28324.252917,2663.873,1.681121e+12,7.546256e+07,27006,1280.419,3.626992e+07,...,28161.072896,28031.051479,27901.030062,-242895.273218,99.996441,27107.166592,0.009050,76.366250,165.339821,-1608825.245
2023-04-10 12:00:00,28311.558750,28317.408750,28306.937500,28312.070417,8308.116,1.681135e+12,2.355780e+08,72121,4324.082,1.226160e+08,...,28172.550312,28048.120312,27923.690312,-243058.255240,100.003555,27107.417294,0.004343,67.411815,132.352424,-1617133.361
2023-04-10 16:00:00,29018.914583,29038.892500,29006.634583,29022.027083,20892.464,1.681150e+12,6.068440e+08,204568,10887.271,3.161619e+08,...,28293.134563,28104.912313,27916.690063,-244012.218223,99.996478,27108.418751,-0.009799,117.929762,315.854958,-1596240.897
2023-04-10 20:00:00,29405.187083,29418.746667,29395.765833,29407.113750,14054.250,1.681164e+12,4.148080e+08,128103,7300.202,2.154915e+08,...,28403.961146,28179.506313,27955.051479,-244186.513959,99.996501,27109.227112,-0.012259,143.694911,301.258480,-1582186.647


In [12]:
merge5.iloc[-10:][['close_min'] + interest_cols_1h_level]

,close_min,open_time,RSI_14_min,RSI_14_5min,RSI_14_10min,RSI_14_15min,RSI_14_30min,RSI_14_hour,volume_min,volume_5min,volume_10min,volume_15min,volume_30min,volume_hour
253,26258.9,2023-03-14 13:00:00,70.579559,87.169851,93.673362,94.737028,90.598297,93.903385,1065.188,4441.179,7075.826,8530.116,14763.027,22807.786
254,25686.7,2023-03-17 02:00:00,73.677843,98.342135,88.061035,89.623646,82.113521,78.610383,406.767,957.246,2957.798,3910.766,5885.097,9153.338
255,28271.0,2023-03-23 15:00:00,87.184487,89.888166,74.586192,75.640565,74.775896,83.449761,1206.796,2894.162,5064.721,7105.827,14473.906,19109.992
256,27928.9,2023-03-26 12:00:00,85.316632,83.731304,82.993632,81.991389,89.658680,85.784557,526.231,1244.199,1579.423,1817.924,2386.374,3266.518
257,28171.7,2023-03-26 14:00:00,79.752066,88.911523,88.001785,87.704448,94.864511,80.772363,182.337,808.296,1027.890,1386.686,2454.018,7231.190
258,26876.6,2023-03-27 15:00:00,10.735703,7.740267,11.244094,2.378916,11.508394,14.841261,969.785,3068.751,5043.125,7201.318,10147.427,12660.227
259,28042.1,2023-03-29 07:00:00,26.834997,86.491497,79.763290,84.635828,86.939161,90.864097,99.817,322.778,580.610,687.200,1284.027,2348.409
260,28160.3,2023-04-08 06:00:00,80.886427,95.001748,79.636477,84.457672,78.249139,81.095593,62.569,245.490,293.329,469.078,764.998,1090.316
261,28484.3,2023-04-09 22:00:00,71.130435,91.679906,90.428782,86.375755,93.282598,87.197991,190.659,712.849,1014.158,1429.260,1898.026,2641.722
262,29113.3,2023-04-10 17:00:00,80.781044,91.127201,90.854939,93.193066,87.143301,87.010771,246.447,1451.655,2451.081,3420.078,5262.591,8266.367


In [13]:
merge6.iloc[-10:][['close_min'] + interest_cols_4h_level]

,close_min,open_time,RSI_14_min,RSI_14_5min,RSI_14_10min,RSI_14_15min,RSI_14_30min,RSI_14_hour,RSI_14_4hour,volume_min,volume_5min,volume_10min,volume_15min,volume_30min,volume_hour,volume_4hour
24,38496.4,2022-05-01 16:00:00,75.639300,88.408207,92.066603,89.037921,80.268801,77.963857,28.598509,39.436,280.607,450.604,658.366,1128.088,1570.145,5782.418
25,31203.5,2022-06-06 04:00:00,72.263288,85.884654,88.232960,94.637740,92.302200,89.035248,86.410358,37.014,744.973,1101.908,1260.574,2127.467,3096.057,10620.252
26,24689.7,2022-06-13 08:00:00,28.804815,28.429402,28.799017,20.938811,24.773022,10.901812,9.227577,337.721,2222.196,3899.915,5119.070,8691.201,19074.872,62756.716
27,21243.1,2022-07-26 00:00:00,19.535415,18.557750,12.312573,12.981456,28.764820,27.155586,28.928618,480.078,1400.676,1872.003,3105.240,6714.942,10776.399,21286.406
28,23177.2,2022-08-19 00:00:00,28.317836,26.859663,20.291371,13.146559,25.614920,17.615441,23.592009,75.525,471.401,1098.759,1791.893,4568.293,6776.013,17284.830
29,19961.0,2022-09-09 04:00:00,88.458498,96.889129,96.944349,94.161336,91.302914,86.195856,86.931806,831.427,6044.540,7967.129,9595.122,13778.037,19212.192,45857.714
30,20118.8,2022-11-08 04:00:00,18.129614,18.680310,13.081676,16.491401,15.012035,17.972730,3.384569,592.178,1492.628,3135.023,9770.664,17842.957,23391.173,47453.749
31,16462.9,2022-11-20 12:00:00,20.038536,21.622770,23.451991,22.595594,20.637116,12.174475,27.365331,218.365,1098.450,1473.599,1991.579,3048.388,4766.436,10828.212
32,17127.3,2023-01-09 00:00:00,89.909183,89.561506,91.565836,91.845467,91.367028,85.361370,90.381962,224.910,677.342,1229.725,1873.997,2855.469,4745.002,12612.863
33,17936.1,2023-01-12 00:00:00,70.454545,88.124054,92.031621,93.981410,97.197430,86.308684,83.841981,141.417,601.116,1337.599,1817.052,2973.145,13180.544,29298.982


In [ ]:
merge7

In [ ]:
merge6[interest_vol_ratios_4h_level].iloc[0].drop(['open_time'])

In [ ]:
my_dict['df_5min']